In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.1,
    max_tokens=100,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
        ('human', 'I want to cook {cuisine} food.')
    ]
)

chef_chain = chef_prompt | chat  # Chain

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
        ("human", "{recipe}")
    ]
)

veg_chef_chain = veg_chef_prompt | chat  # Chain

# Runnable Map + Chaining between Chain and Chain
final_chain = {"recipe": chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine": 'Classical Italian Carbonara'
})

AIMessage(content='Certainly! To make a vegetarian version of classic Italian Carbonara, you\'ll want to replace the pancetta or guanciale with a flavorful vegetarian alternative. Here are some options and how to prepare them:\n\n**Vegetarian "Pancetta" or "Guanciale" Substitute:**\n- **Smoked Tempeh or Tofu:** Cut into small dice and sauté until crispy.\n- **Mushrooms (e.g., shiitake or portobello):** Slice thinly and cook')